In [1]:
import os
import json
import openai
from openai import AzureOpenAI

In [2]:
AZURE_OPENAI_API_VERSION='2025-01-01-preview'
AZURE_OPENAI_EMBEDDINGS_API_VERSION='2023-05-15'

In [3]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=AZURE_OPENAI_API_VERSION
)

In [4]:
CHAT_DEPLOYMENT = "gpt-5-chat"             
EMBEDDING_DEPLOYMENT = "text-embedding-3-small"

In [20]:
def generate_caption_and_tags(vision_summary_json):

    system_prompt = (
        "Jesteś asystentem generującym krótki opis zdjęcia i listę tagów w języku polskim.\n"
        "Weź pod uwagę wykryte etykiety, opis i tekst (OCR) dostarczony poniżej.\n"
        "Opis zdjęcia powinien być utrzymany w dziennikarskim stylu i nadawać się do publikacji (1-2 zdania). Nie może być suchym opisem widoku\n"
        "Nie opisuj oczywistych elementów widocznych na zdjęciu (kolorów i kształtów); unikaj sformułowań typu 'scena oddaje...', 'na zdjęciu widać...'. Odwołaj się do kontekstu i ogólnej wiedzy o widocznym zjawisku, możesz przywołać ogólne prawdy i znane fakty związane z tematem.\n"
        "Zwróć w formacie JSON z kluczami: 'caption' (1-2 zdania), "
        "'tags' (lista obiektów {tag, confidence}).\n"
        "Użyj naturalnego języka polskiego."
    )

    user_prompt = f"Vision JSON: {vision_summary_json}"

    response = client.chat.completions.create(
        model=CHAT_DEPLOYMENT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=300,
        temperature=0.2,
    )

    text = response.choices[0].message.content

    # Expected JSON output
    try:
        return json.loads(text)
    except Exception:
        return {"raw": text}

In [10]:
sample_json = {'dense_captions': [{'text': 'a group of people holding signs', 'confidence': 0.9424399137496948, 'bbox': {'x': 0, 'y': 0, 'w': 1000, 'h': 580}}, {'text': 'a woman taking a selfie', 'confidence': 0.9134508371353149, 'bbox': {'x': 783, 'y': 188, 'w': 209, 'h': 369}}, {'text': 'a man holding a cellphone', 'confidence': 0.7785478234291077, 'bbox': {'x': 384, 'y': 158, 'w': 233, 'h': 348}}, {'text': 'a man in a blue suit', 'confidence': 0.7847445607185364, 'bbox': {'x': 79, 'y': 192, 'w': 294, 'h': 269}}, {'text': 'a woman holding a sign', 'confidence': 0.8923084735870361, 'bbox': {'x': 533, 'y': 140, 'w': 217, 'h': 280}}, {'text': 'a man with a beard', 'confidence': 0.811130702495575, 'bbox': {'x': 0, 'y': 122, 'w': 143, 'h': 224}}, {'text': 'a man in a hat and leather jacket', 'confidence': 0.6910561919212341, 'bbox': {'x': 830, 'y': 31, 'w': 163, 'h': 269}}, {'text': 'a woman speaking into a microphone', 'confidence': 0.7274485230445862, 'bbox': {'x': 242, 'y': 149, 'w': 148, 'h': 322}}, {'text': 'a person holding a sign', 'confidence': 0.9028834700584412, 'bbox': {'x': 530, 'y': 65, 'w': 194, 'h': 148}}, {'text': 'a blurry image of a blue sign', 'confidence': 0.7562562823295593, 'bbox': {'x': 773, 'y': 1, 'w': 172, 'h': 59}}], 'tags': [{'tag': 'clothing', 'confidence': 0.9988632798194885}, {'tag': 'human face', 'confidence': 0.9978727102279663}, {'tag': 'person', 'confidence': 0.9968821406364441}, {'tag': 'woman', 'confidence': 0.9683936834335327}, {'tag': 'text', 'confidence': 0.956762969493866}, {'tag': 'man', 'confidence': 0.9565277099609375}, {'tag': 'smile', 'confidence': 0.9443103075027466}, {'tag': 'newspaper', 'confidence': 0.8428946137428284}, {'tag': 'outdoor', 'confidence': 0.8321356773376465}, {'tag': 'group', 'confidence': 0.7874609231948853}, {'tag': 'sign', 'confidence': 0.7766597867012024}, {'tag': 'people', 'confidence': 0.7492111921310425}], 'landmarks': [], 'ocr_text': 'TRUMI\nJUMP\nMAKE AMERICA GREAT AGATE\nTRUMP\nthe silent majority\nST\nT\nSTANDS WITH\nSUME\nBREAMERICA GREAT AGAIN!\nTRUMP\nAN\nS\nTR\nRUMP\nthe sites\nTRUMP\nS W\nAMERI GREAT AGAIN!\nMAKE AMERICA GREAT AGAIN!\nRUMP\nthe ailem matarits\nSTANDS WITH'}

In [11]:
sample_json2 = {'dense_captions': [{'text': 'a group of jockeys riding horses on grass', 'confidence': 0.8134883046150208, 'bbox': {'x': 0, 'y': 0, 'w': 1920, 'h': 1079}}, {'text': 'a jockey riding a horse', 'confidence': 0.7614299058914185, 'bbox': {'x': 809, 'y': 224, 'w': 465, 'h': 727}}, {'text': 'a group of jockeys riding horses on grass', 'confidence': 0.812589168548584, 'bbox': {'x': 62, 'y': 100, 'w': 1769, 'h': 921}}, {'text': 'a group of horses on a track', 'confidence': 0.6950061321258545, 'bbox': {'x': 300, 'y': 148, 'w': 625, 'h': 885}}, {'text': 'a jockey on a horse', 'confidence': 0.7583951354026794, 'bbox': {'x': 440, 'y': 10, 'w': 311, 'h': 544}}, {'text': 'a jockey riding a horse', 'confidence': 0.7707228064537048, 'bbox': {'x': 64, 'y': 527, 'w': 310, 'h': 409}}, {'text': 'a jockey riding a horse in a race', 'confidence': 0.7399764060974121, 'bbox': {'x': 1181, 'y': 171, 'w': 555, 'h': 782}}, {'text': 'a jockey on a horse', 'confidence': 0.7769850492477417, 'bbox': {'x': 873, 'y': 137, 'w': 274, 'h': 457}}, {'text': 'a person wearing a white hat', 'confidence': 0.7403091788291931, 'bbox': {'x': 252, 'y': 476, 'w': 72, 'h': 66}}, {'text': 'a green and yellow hat with a pompom', 'confidence': 0.8064011335372925, 'bbox': {'x': 1049, 'y': 152, 'w': 100, 'h': 89}}], 'tags': [{'tag': 'horse', 'confidence': 0.9770833253860474}, {'tag': 'stadium', 'confidence': 0.9752413034439087}, {'tag': 'outdoor', 'confidence': 0.9523265957832336}, {'tag': 'building', 'confidence': 0.9427610039710999}, {'tag': 'rein', 'confidence': 0.9391779899597168}, {'tag': 'bridle', 'confidence': 0.9323909282684326}, {'tag': 'stallion', 'confidence': 0.9239025712013245}, {'tag': 'horse tack', 'confidence': 0.9214749336242676}, {'tag': 'horse supplies', 'confidence': 0.9210265874862671}, {'tag': 'flat racing', 'confidence': 0.9156216979026794}, {'tag': 'halter', 'confidence': 0.9028061628341675}, {'tag': 'mare', 'confidence': 0.8987149596214294}, {'tag': 'equestrianism', 'confidence': 0.8742722868919373}, {'tag': 'saddle', 'confidence': 0.8724848031997681}, {'tag': 'horse racing', 'confidence': 0.870876133441925}, {'tag': 'jockey', 'confidence': 0.8687554597854614}, {'tag': 'animal sports', 'confidence': 0.8578435778617859}, {'tag': 'grass', 'confidence': 0.8521130084991455}, {'tag': 'mane', 'confidence': 0.8485252857208252}, {'tag': 'racing', 'confidence': 0.8210968375205994}, {'tag': 'person', 'confidence': 0.6902799606323242}, {'tag': 'riding', 'confidence': 0.6800144910812378}, {'tag': 'people', 'confidence': 0.5848963856697083}, {'tag': 'track', 'confidence': 0.5521560311317444}], 'landmarks': [], 'ocr_text': 'oreira\nMELIS EXPRES\nSRISIG\nLONO REF.'}

In [24]:
sample_json3 = {'dense_captions': [{'text': 'a man in a suit holding up his hand', 'confidence': 0.8234245181083679, 'bbox': {'x': 0, 'y': 0, 'w': 960, 'h': 1165}}, {'text': 'a man in a suit holding up his hand', 'confidence': 0.8233250379562378, 'bbox': {'x': 21, 'y': 43, 'w': 916, 'h': 1097}}, {'text': "close-up of a black and white photo of a man's head", 'confidence': 0.7811080813407898, 'bbox': {'x': 522, 'y': 476, 'w': 162, 'h': 351}}, {'text': 'a man in a suit and tie', 'confidence': 0.8933911323547363, 'bbox': {'x': 204, 'y': 458, 'w': 670, 'h': 696}}, {'text': 'close-up of a man with a mustache', 'confidence': 0.9058632850646973, 'bbox': {'x': 372, 'y': 72, 'w': 289, 'h': 391}}, {'text': 'a black and white photo of a lamp', 'confidence': 0.7153570652008057, 'bbox': {'x': 826, 'y': 167, 'w': 129, 'h': 528}}, {'text': "a close-up of a man's lips", 'confidence': 0.8710940480232239, 'bbox': {'x': 476, 'y': 322, 'w': 114, 'h': 79}}, {'text': 'a close-up of a hand', 'confidence': 0.8892737030982971, 'bbox': {'x': 79, 'y': 55, 'w': 202, 'h': 292}}, {'text': "a close up of a person's nose", 'confidence': 0.8483901619911194, 'bbox': {'x': 490, 'y': 262, 'w': 88, 'h': 65}}], 'tags': [{'tag': 'person', 'confidence': 0.9959255456924438}, {'tag': 'human face', 'confidence': 0.9956406354904175}, {'tag': 'clothing', 'confidence': 0.9908991456031799}, {'tag': 'black and white', 'confidence': 0.9600116610527039}, {'tag': 'man', 'confidence': 0.9584307670593262}, {'tag': 'portrait', 'confidence': 0.8911531567573547}, {'tag': 'smile', 'confidence': 0.8660576343536377}, {'tag': 'wall', 'confidence': 0.8595969080924988}, {'tag': 'gentleman', 'confidence': 0.846953272819519}, {'tag': 'indoor', 'confidence': 0.8446080684661865}, {'tag': 'black', 'confidence': 0.7384896278381348}, {'tag': 'wearing', 'confidence': 0.7249739766120911}, {'tag': 'suit', 'confidence': 0.7116806507110596}], 'landmarks': []}

In [21]:
generate_caption_and_tags(sample_json)

{'raw': '```json\n{\n  "caption": "Zwolennicy Donalda Trumpa gromadzą się na wiecu, trzymając transparenty z hasłami kampanii i robiąc zdjęcia w tłumie. Spotkania tego typu od lat stanowią ważny element amerykańskiej sceny politycznej, łącząc entuzjazm wyborców z atmosferą medialnego spektaklu.",\n  "tags": [\n    {"tag": "wiec polityczny", "confidence": 0.95},\n    {"tag": "Donald Trump", "confidence": 0.94},\n    {"tag": "kampania wyborcza", "confidence": 0.93},\n    {"tag": "protest", "confidence": 0.88},\n    {"tag": "transparent", "confidence": 0.87},\n    {"tag": "tłum", "confidence": 0.85},\n    {"tag": "USA", "confidence": 0.84},\n    {"tag": "polityka", "confidence": 0.83},\n    {"tag": "zwolennicy", "confidence": 0.82},\n    {"tag": "wydarzenie publiczne", "confidence": 0.8}\n  ]\n}\n```'}

In [22]:
generate_caption_and_tags(sample_json2)

{'raw': '```json\n{\n  "caption": "Zawodowi dżokeje rywalizują na torze wyścigowym, gdzie precyzja i refleks decydują o ułamkach sekund różnicy między zwycięstwem a porażką. Wyścigi konne pozostają jednym z najbardziej widowiskowych sportów, łącząc tradycję z nowoczesnym treningiem zwierząt i jeźdźców.",\n  "tags": [\n    {"tag": "wyścigi konne", "confidence": 0.87},\n    {"tag": "dżokej", "confidence": 0.86},\n    {"tag": "sport", "confidence": 0.85},\n    {"tag": "koń", "confidence": 0.97},\n    {"tag": "tor wyścigowy", "confidence": 0.82},\n    {"tag": "jeździectwo", "confidence": 0.87},\n    {"tag": "zawody sportowe", "confidence": 0.84},\n    {"tag": "stadion", "confidence": 0.97},\n    {"tag": "rywalizacja", "confidence": 0.81},\n    {"tag": "na świeżym powietrzu", "confidence": 0.95}\n  ]\n}\n```'}

In [25]:
generate_caption_and_tags(sample_json3)

{'caption': 'Elegancko ubrany mężczyzna w geście powitania lub przysięgi przyciąga uwagę klasycznym, czarno-białym portretem, przypominającym fotografie z czasów, gdy formalność i gest miały szczególne znaczenie w komunikacji publicznej.',
 'tags': [{'tag': 'portret', 'confidence': 0.89},
  {'tag': 'mężczyzna', 'confidence': 0.96},
  {'tag': 'czarno-białe zdjęcie', 'confidence': 0.96},
  {'tag': 'garnitur', 'confidence': 0.71},
  {'tag': 'gest', 'confidence': 0.82},
  {'tag': 'elegancja', 'confidence': 0.84},
  {'tag': 'fotografia studyjna', 'confidence': 0.85}]}

In [16]:
def get_embedding(text: str):
    emb = client.embeddings.create(
        model=EMBEDDING_DEPLOYMENT,
        input=text
    )
    return emb.data[0].embedding

In [18]:
test_embedding = get_embedding('Na torze wyścigowym kilku dżokejów rywalizuje w dynamicznym biegu konnym')

In [19]:
len(test_embedding)

1536

In [20]:
embed_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=AZURE_OPENAI_EMBEDDINGS_API_VERSION
)

In [21]:
def get_embedding2(text: str):
    emb = embed_client.embeddings.create(
        model=EMBEDDING_DEPLOYMENT,
        input=text
    )
    return emb.data[0].embedding

In [22]:
test_embedding2 = get_embedding2('Na torze wyścigowym kilku dżokejów rywalizuje w dynamicznym biegu konnym')

In [23]:
test_embedding == test_embedding2

True